In [64]:
import csv
import sys
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

EMPTY_LIST  = []
TITLE = 1
SUBTITLE = 2
CONTENT = 3
URL = 4
ID = 5

tokenizer = RegexpTokenizer(r'\w+')

### Item 1 - Escreva uma função que receba uma coleção de documentos e retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas no texto (bigramas).

Primeiramente escreveremos uma função que retorna uma lista em que cada elemento é uma palavra de um documento do corpus. Para melhorar o processo iremos utilizar o stopwords do nltk para eliminar termos indesejados tais como conjunções, preposições e etc.

In [65]:

def getWords():
    data_frame = pd.read_csv('estadao_noticias_eleicao.csv')
    data = (data_frame.titulo + " " + data_frame.subTitulo + " " + data_frame.conteudo).fillna("")
    stopword_ = stopwords.words('portuguese')
    data = data \
            .apply( \
                    lambda text: \
                        tokenizer.tokenize(text.lower()) \
            ).apply( \
                    lambda tokens: \
                        [d for d in tokens if d not in stopword_] \
            )
    data = [ el for row in data for el in row ]
    return data

Agora utilizaremos o código disponibilizado [aqui](https://github.com/allansales/information-retrieval/blob/master/Lab%202/coocurrence_matrix.ipynb) para obter nossa matriz de termos-termos com a frequência de bigramas em todo o corpus.

In [66]:
from co_ocurrence import co_occurrence_matrix

### Item 2 - Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.

In [67]:
def top_n(word, n):
    global term_term_matrix, vocab
    word = word.lower()
    if word not in vocab.keys(): 
        return []
    matches = [[w1, usable_matrix[vocab[word], vocab[w1]]] for w1 in vocab if usable_matrix[vocab[word], vocab[w1]] > 0]
    result = sorted(matches, key= lambda x:x[1], reverse=True)
    return result[:n]

words = readData()
term_term_matrix, vocab = co_occurrence_matrix(words)
consultable_matrix = term_term_matrix.tocsr()

query_1 = "corrupção"
query_2 = "dinheiro"
query_3 = "governo"

print("palavras que mais co-ocorrem com {0} = {1}".format(query_1, top_n(query_1, 3))) 
print("palavras que mais co-ocorrem com {0} = {1}".format(query_2, top_n(query_2, 3)))
print("palavras que mais co-ocorrem com {0} = {1}".format(query_3, top_n(query_3, 3)))
    

palavras que mais co-ocorrem com corrupção = [['petrobrás', 109], ['ativa', 53], ['passiva', 51]]
palavras que mais co-ocorrem com dinheiro = [['público', 105], ['é', 28], ['vivo', 26]]
palavras que mais co-ocorrem com governo = [['federal', 672], ['dilma', 514], ['estado', 314]]


### Item 3 - Expanda a consulta original com os termos retornados no passo 2 acima.

In [93]:
from itertools import chain

def getWords(top_n_list):
    return [word_frequency[0] for word_frequency in top_n_list]

def expandQuery(query):
    newQuery = list(chain.from_iterable([t, *getWords(top_n(t, 3))]for t in tokenizer.tokenize(query.lower())))
    return " ".join(set(newQuery))

expandQuery("governo")

'estado governo dilma federal'